## Synthetic spectra generator

In [1]:
#First reset sys path so we use the correct envoirnment
import sys
sys.path=['', 'C:\\Users\\bryan\\Anaconda3\\envs\\tensorflow_GPU\\python37.zip', 'C:\\Users\\bryan\\Anaconda3\\envs\\tensorflow_GPU\\DLLs', 'C:\\Users\\bryan\\Anaconda3\\envs\\tensorflow_GPU\\lib', 'C:\\Users\\bryan\\Anaconda3\\envs\\tensorflow_GPU', 'C:\\Users\\bryan\\AppData\\Roaming\\Python\\Python37\\site-packages', 'C:\\Users\\bryan\\Anaconda3\\envs\\tensorflow_GPU\\lib\\site-packages']

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
n_points = 1000

In [2]:
import tensorflow as tf
import keras.backend as K
from keras.models import Model, Sequential, load_model
from keras.layers import Dense, Conv1D, Flatten, BatchNormalization, Activation, Dropout
from keras import regularizers
from datetime import datetime

In [3]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [4]:
tf.keras.backend.clear_session()
model = Sequential()

model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None,input_shape = (n_points, 1)))

model.add(Activation('relu'))

model.add(Conv1D(128, activation = 'relu', kernel_size = (32)))
model.add(Conv1D(64, activation = 'relu', kernel_size = (16)))
model.add(Conv1D(16, activation = 'relu', kernel_size = (8)))
model.add(Conv1D(16, activation = 'relu', kernel_size = (8)))
model.add(Conv1D(16, activation = 'relu', kernel_size = (8)))
model.add(Dense(32, activation = 'relu', kernel_regularizer=regularizers.l1_l2(l1 = 0, l2=0.1)))
model.add(Dense(16, activation = 'relu', kernel_regularizer=regularizers.l1_l2(l1 = 0, l2=0.1)))
model.add(Flatten())
model.add(Dropout(.25))
model.add(Dense(n_points, activation='relu'))


model.compile(loss='mse', optimizer='Adam', metrics=['mean_absolute_error','mse','accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 1000, 1)           4         
_________________________________________________________________
activation (Activation)      (None, 1000, 1)           0         
_________________________________________________________________
conv1d (Conv1D)              (None, 969, 128)          4224      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 954, 64)           131136    
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 947, 16)           8208      
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 940, 16)           2064      
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 933, 16)           2

## Training

In [ ]:
from Generate_Data import *
for dataset_num in range(3,10):
    if dataset_num == 1:
        a=1
        b='a'
    elif dataset_num == 2:
        a=1
        b='b'
    elif dataset_num == 3:
        a=1
        b='c'
    elif dataset_num == 4:
        a=2
        b='a'
    elif dataset_num == 5:
        a=2
        b='b'
    elif dataset_num == 6:
        a=2
        b='c'
    elif dataset_num == 7:
        a=3
        b='a'
    elif dataset_num == 8:
        a=3
        b='b'
    else:
        a=3
        b='c'
    X, y = generate_datasets_for_Paper_1(dataset_num,220000)
    history = model.fit(X, y,epochs=10, verbose = 1, validation_split=0.090909091, batch_size=256) 
    my_path='./data/'+str(a)+b+'PAPER1'
    model.save(my_path)
    hist_df = pd.DataFrame(history.history) 
    with open(my_path+'.csv', mode='w') as f:
        hist_df.to_csv(f)
    
    
    my_path='./data/'+str(a)+b+'PAPER1'
    #history = load_model(my_path)
    # plt.plot(history.history['loss']) 
    # plt.plot(history.history['val_loss'])
    # plt.title('Model loss')
    # plt.ylabel('Loss')
    # plt.xlabel('Epoch')
    # plt.legend(['Train', 'Test'], loc='upper left')
    # plt.show()


    size1=100
    name1='./data/' +str(a)+b+'Raman_spectrums_valid.csv'
    name2= './data/'+str(a)+b+'CARS_spectrums_valid.csv'
    RAMAN = pd.read_csv(name1)
    BCARS = pd.read_csv(name2)
    RAMAN = RAMAN.values[:,1:]
    BCARS = BCARS.values[:,1:]
    RESULTS = RAMAN[:,:]
    print(np.shape(BCARS))
    X = np.empty((1, n_points,1))

    for m in range(size1):
        X[0,:,0]=BCARS[m,:]
        yhat = model.predict(X, verbose =0)
        RESULTS[m,:]=yhat.flatten()
        #print(RESULTS[m,:])

    #    print(np.shape(yhat))
    #     f, a = plt.subplots(2,1, sharex=True)
    #     a[0].plot(X.flatten(), label = 'cars')
    #     a[1].plot(RAMAN[m,:].T+.7, label = 'true',c= 'g' )
    #     a[1].plot(yhat.flatten()+1.4, label = 'pred.',c='r')
    #     plt.subplots_adjust(hspace=0)

    pd.DataFrame(RESULTS).to_csv('./data/' +str(a)+b+'Raman_spectrums_valid_PAPER1.csv')


Epoch 1/10
782/782 [==============================] - 39s 50ms/step - loss: 0.5707 - mean_absolute_error: 0.0225 - mse: 0.0019 - accuracy: 0.3288 - val_loss: 0.0011 - val_mean_absolute_error: 0.0143 - val_mse: 7.8602e-04 - val_accuracy: 0.5368
Epoch 2/10
782/782 [==============================] - 40s 52ms/step - loss: 6.9129e-04 - mean_absolute_error: 0.0129 - mse: 5.8534e-04 - accuracy: 0.4850 - val_loss: 4.6744e-04 - val_mean_absolute_error: 0.0112 - val_mse: 3.9700e-04 - val_accuracy: 0.6024
Epoch 3/10
782/782 [==============================] - 41s 52ms/step - loss: 5.3578e-04 - mean_absolute_error: 0.0117 - mse: 4.7730e-04 - accuracy: 0.5273 - val_loss: 3.5663e-04 - val_mean_absolute_error: 0.0095 - val_mse: 3.0483e-04 - val_accuracy: 0.6491
Epoch 4/10
782/782 [==============================] - 41s 52ms/step - loss: 4.7192e-04 - mean_absolute_error: 0.0111 - mse: 4.2544e-04 - accuracy: 0.5518 - val_loss: 4.0154e-04 - val_mean_absolute_error: 0.0110 - val_mse: 3.5689e-04 - val_accur

Epoch 6/10
782/782 [==============================] - 40s 52ms/step - loss: 3.1472e-04 - mean_absolute_error: 0.0095 - mse: 2.9868e-04 - accuracy: 0.6703 - val_loss: 2.8008e-04 - val_mean_absolute_error: 0.0089 - val_mse: 2.6494e-04 - val_accuracy: 0.8559
Epoch 7/10
782/782 [==============================] - 40s 52ms/step - loss: 3.1049e-04 - mean_absolute_error: 0.0094 - mse: 2.9463e-04 - accuracy: 0.6721 - val_loss: 2.5381e-04 - val_mean_absolute_error: 0.0085 - val_mse: 2.3941e-04 - val_accuracy: 0.8546
Epoch 8/10
782/782 [==============================] - 40s 52ms/step - loss: 3.0558e-04 - mean_absolute_error: 0.0093 - mse: 2.9047e-04 - accuracy: 0.6744 - val_loss: 2.6398e-04 - val_mean_absolute_error: 0.0087 - val_mse: 2.5007e-04 - val_accuracy: 0.8597
Epoch 9/10
782/782 [==============================] - 40s 52ms/step - loss: 3.9892e-04 - mean_absolute_error: 0.0101 - mse: 3.5287e-04 - accuracy: 0.6669 - val_loss: 2.1298e-04 - val_mean_absolute_error: 0.0079 - val_mse: 1.9789e-04

Use this function to test the model on single instances

In [ ]:
# def predict_and_plot():
#     x,y = generate_one_spectrum_Paper_1(dataset_num)
#     yhat = model.predict(x, verbose =0)
#     f, a = plt.subplots(2,1, sharex=True)
#     a[0].plot(x.flatten(), label = 'cars')
#     a[1].plot(y.T+.7, label = 'true',c= 'g' )
#     a[1].plot(yhat.flatten()+1.4, label = 'pred.',c='r')
#     plt.subplots_adjust(hspace=0)
#     #return x, y.flatten(), yhat.flatten(), chi3, NRB

In [ ]:
# #predict_and_plot()
# for dataset_num in range(1,10):
#     if dataset_num == 1:
#         a=1
#         b='a'
#     elif dataset_num == 2:
#         a=1
#         b='b'
#     elif dataset_num == 3:
#         a=1
#         b='c'
#     elif dataset_num == 4:
#         a=2
#         b='a'
#     elif dataset_num == 5:
#         a=2
#         b='b'
#     elif dataset_num == 6:
#         a=2
#         b='c'
#     elif dataset_num == 7:
#         a=3
#         b='a'
#     elif dataset_num == 8:
#         a=3
#         b='b'
#     else:
#         a=3
#         b='c'
#     my_path='./data/'+str(a)+b+'PAPER1'
#     history = load_model(my_path)
#     # plt.plot(history.history['loss']) 
#     # plt.plot(history.history['val_loss'])
#     # plt.title('Model loss')
#     # plt.ylabel('Loss')
#     # plt.xlabel('Epoch')
#     # plt.legend(['Train', 'Test'], loc='upper left')
#     # plt.show()


#     size1=100
#     name1='./data/' +str(a)+b+'Raman_spectrums_train.csv'
#     name2= './data/'+str(a)+b+'CARS_spectrums_train.csv'
#     RAMAN = pd.read_csv(name1)
#     BCARS = pd.read_csv(name2)
#     RAMAN = RAMAN.values[:,1:]
#     BCARS = BCARS.values[:,1:]
#     RESULTS = RAMAN[:,:]
#     print(np.shape(BCARS))
#     X = np.empty((1, n_points,1))

#     for m in range(size1):
#         X[0,:,0]=BCARS[m,:]
#         yhat = model.predict(X, verbose =0)
#         RESULTS[m,:]=yhat.flatten()
#         #print(RESULTS[m,:])

#     #    print(np.shape(yhat))
#     #     f, a = plt.subplots(2,1, sharex=True)
#     #     a[0].plot(X.flatten(), label = 'cars')
#     #     a[1].plot(RAMAN[m,:].T+.7, label = 'true',c= 'g' )
#     #     a[1].plot(yhat.flatten()+1.4, label = 'pred.',c='r')
#     #     plt.subplots_adjust(hspace=0)

#     pd.DataFrame(RESULTS).to_csv('./data/' +str(a)+b+'Raman_spectrums_valid_PAPER1.csv')